Flux with MNIST and GPU


Libraries

In [ ]:
using CUDA

using MLDatasets           # mnist
using PreprocessingImages; pim = PreprocessingImages

using MLDataUtils          # label, nlabel, labelfreq, stratifiedobs
using Flux                 # the julia ml library

using Random

In [ ]:
include( expanduser("~/projects/pesquisa/libs/libml.jl") )

MNIST

In [ ]:
# load mnist
trX,   trY   = MNIST(:train)[:]
testX, testY = MNIST(:test)[:];

In [ ]:
# split train, validation, test sets
Random.seed!(1)
(trainX, trainY), (validationX, validationY) = stratifiedobs((trX, trY), p = 0.7)
size(trainX), size(validationX), size(testX)

Preprocessing

Flux primarily works with matrices and vectors

In [ ]:
function preprocess(X, y)
    Xs = permutedims(X, (2, 1, 3))
    Xs = Xs |> Flux.flatten
    
    ys = Flux.onehotbatch( Float32.(y), 0:9 )
    
    return (Xs, ys)
end

h, v, N = size(trainX); d = h * v
X_c, y_c = preprocess(trainX, trainY)
typeof(X_c), size(X_c), size(y_c)

Model

In [ ]:
# data on gpu
X_g = X_c |> gpu
y_g = y_c |> gpu
typeof(X_g), size(X_g), size(y_g)

In [ ]:
# model configuration
nInputs  = d
nOutputs = 10

# Flux.Dense(nInputs, 256, tanh)
model = Chain( Dense(nInputs => 128, relu),
               Dense(128     => 32,  relu),
               Dense(32      => nOutputs),
               softmax)      |> gpu           # weights on gpu

In [ ]:
lossFunction(X, y) = Flux.mse( model(X), y )
modelParameters    = Flux.params(model)
dataset            = Flux.DataLoader((X_g, y_g), batchsize=32)       # batchsize=1 => SGD, batch size > 1 => batch gradient descent
modelOptimizer     = Flux.Descent()
callBack           = Flux.throttle(() -> println("."), 10);   # print every 10s

Training

In [ ]:
numberOfEpochs = 5

Training method without plotting epochs

In [ ]:
# # https://fluxml.ai/Flux.jl/stable/training/training/

# # preferred for multiple epochs
# epochs = 1:numberOfEpochs
# for epoch in epochs
#     Flux.train!(lossFunction, modelParameters, data, modelOptimizer; cb=callBack)
# end

Training epoch with plotting, with the aid of homemade function trainModel!

In [ ]:
# https://fluxml.ai/Flux.jl/stable/training/training/

epochs     = 1 : numberOfEpochs
lossVector = Vector{Float64}()
minLoss    = 1e-6   # hint: start at 1e-2, then go lower for more epochs (5e-4)
nearZero   = minLoss / 25

for epoch in epochs
    # train epoch
    loss = trainModel!( lossFunction, modelParameters, dataset, modelOptimizer )   # libml
    push!(lossVector, loss)
    print(".")   # "epoch completed" indicator

    # exit criteria
    if stopTrainingCriteria(lossVector, minLoss, nearZero)   break   end
end

In [ ]:
# plot training
plotLoss(lossVector, (500,300))

Validation

In [ ]:
function predictOutcome(X)
    ŷ = Flux.onecold( model(X), [0:9;] )
end

In [ ]:
# preprocessing
X_c, y_c = preprocess(validationX, validationY)
X_g      = X_c |> gpu

# predict
ŷ = predictOutcome(X_g)

# result
printMetrics( ŷ, coerce(validationY, OrderedFactor) )